In [40]:
import os
import urllib.request
import zipfile
import matplotlib.pyplot as plt
import seaborn as sns

import geopandas as gpd
import pandas as pd
import folium
from branca.colormap import LinearColormap
from streamlit_folium import folium_static as fs, st_folium as stf


In [41]:
def load_state_map(data_dir: str, eligibility_df: pd.DataFrame) -> folium.Map:
    print(eligibility_df.isnull().values.any())
    
    eligibility_df["Total Change Percentage Eligible"] = (eligibility_df["Current Percentage Eligible"] -
                                                          eligibility_df["New Percentage Eligible"])

    columns = eligibility_df.columns.tolist()

    columns.remove("state")
    columns.insert(0, "NAME")

    eligibility_df["state"] = eligibility_df["state"].astype(str).str.zfill(2)

    state_shapefile = data_dir + 'ShapeFiles/States/tl_2022_us_state.shp'

    # Read the Shapefile
    gdf = gpd.read_file(state_shapefile)

    merged_data = gdf.merge(eligibility_df, left_on='GEOID', right_on='state', how='left')

    min_value = merged_data['Total Change Percentage Eligible'].min()
    max_value = merged_data['Total Change Percentage Eligible'].max()
    
    print(min_value, max_value)
    
    # Print if there are any null values
    print(merged_data['Total Change Percentage Eligible'].isnull().values.any())
    
    # Print all the rows where total change percentage eligible is null
    print(len(merged_data[merged_data['Total Change Percentage Eligible'].isnull()]))
    print(merged_data[merged_data['Total Change Percentage Eligible'].isnull()])

    cmap = sns.color_palette("coolwarm", as_cmap=True)  # You can choose a different palette
    norm = plt.Normalize(vmin=min_value, vmax=max_value)

    def color_function(feature):
        value = feature['properties']['Total Change Percentage Eligible']
        color = cmap(norm(value))
        return f"rgb({int(color[0] * 255)}, {int(color[1] * 255)}, {int(color[2] * 255)})"


    # Create a map without specifying a center or zoom level
    m = folium.Map([37.090240, -95.712891], zoom_start=4)

    folium.GeoJson(
        merged_data,
        style_function=lambda feature: {
            'fillColor': color_function(feature),
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.6
        },
        tooltip=folium.GeoJsonTooltip(fields=['NAME'], aliases=['State'], sticky=True),
        popup=folium.GeoJsonPopup(fields=columns, localize=True),
    ).add_to(m)

    return m



In [42]:
df = pd.read_csv("percentage_eligible_has_pap_has_ssip_has_hins4_black-state.csv")

m = load_state_map("../Data/", df)


False
0.21000000000000085 1.8100000000000023
True
5
   REGION DIVISION STATEFP   STATENS GEOID STUSPS  \
34      9        0      78  01802710    78     VI   
35      9        0      69  01779809    69     MP   
36      9        0      66  01802705    66     GU   
41      9        0      60  01802701    60     AS   
53      3        5      11  01702382    11     DC   

                                            NAME LSAD  MTFCC FUNCSTAT  \
34                  United States Virgin Islands   00  G4000        A   
35  Commonwealth of the Northern Mariana Islands   00  G4000        A   
36                                          Guam   00  G4000        A   
41                                American Samoa   00  G4000        A   
53                          District of Columbia   00  G4000        A   

        ALAND      AWATER     INTPTLAT      INTPTLON  \
34  348021909  1550236187  +18.3392359  -064.9500433   
35  472292521  4644252458  +15.0010865  +145.6181702   
36  543555849   934337

In [46]:
metdiv = gpd.read_file(f"zip://../Data/ShapeFiles/COUNTY/tl_2022_us_county.zip!tl_2022_us_county.shp")

print(metdiv.head())

  STATEFP COUNTYFP  COUNTYNS  GEOID       NAME          NAMELSAD LSAD CLASSFP  \
0      31      039  00835841  31039     Cuming     Cuming County   06      H1   
1      53      069  01513275  53069  Wahkiakum  Wahkiakum County   06      H1   
2      35      011  00933054  35011    De Baca    De Baca County   06      H1   
3      31      109  00835876  31109  Lancaster  Lancaster County   06      H1   
4      31      129  00835886  31129   Nuckolls   Nuckolls County   06      H1   

   MTFCC  CSAFP  CBSAFP  METDIVFP FUNCSTAT       ALAND    AWATER     INTPTLAT  \
0  G4020    NaN     NaN       NaN        A  1477644346  10691216  +41.9158651   
1  G4020    NaN     NaN       NaN        A   680980770  61564427  +46.2946377   
2  G4020    NaN     NaN       NaN        A  6016818946  29090018  +34.3592729   
3  G4020    NaN     NaN       NaN        A  2169272978  22847034  +40.7835474   
4  G4020    NaN     NaN       NaN        A  1489645185   1718484  +40.1764918   

       INTPTLON           